# 3C structure

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker  # 用于设置刻度格式

# ------------------ 设置风格 ------------------ #
sns.set_style("whitegrid")
plt.rcParams.update({
    "axes.edgecolor": "black",     
    "axes.linewidth": 1.0,         
    "axes.facecolor": "white",     
    "grid.color": "gray",          
    "grid.alpha": 0.2,             
    "grid.linestyle": "-",
    "xtick.color": "black",
    "ytick.color": "black",
    "text.color": "black",
    "font.size": 12,
    "figure.facecolor": "white",
})
sns.set_context("notebook", font_scale=1.2)

# 使用 GnBu 渐变色调色板
cmap = sns.color_palette("GnBu", as_cmap=True)

# ------------------ 读取 Excel 文件 ------------------ #
df = pd.read_excel("./vis/performance/structure.xlsx", index_col=0)
df = df.astype(float)

# ------------------ 设置每个单元格大小 ------------------ #
cell_width = 2.0  
cell_height = 2.0  
fig_width = df.shape[1] * cell_width
fig_height = df.shape[0] * cell_height

# ------------------ 绘制热图 ------------------ #
plt.figure(figsize=(fig_width, fig_height))
heatmap = sns.heatmap(
    df,
    annot=True,
    fmt=".2f",             # 保留两位小数
    cmap=cmap,
    square=True,
    linewidths=0.5,
    linecolor='black',
    annot_kws={"fontsize": 16, "ha": "center", "va": "center"}  # 显式指定注释居中
)

# 设置坐标轴刻度标签的旋转和大小
plt.xticks(rotation=45, ha='right', fontsize=18)
plt.yticks(rotation=0, fontsize=18)

# ------------------ 设置色条刻度大小与格式 ------------------ #
cbar = heatmap.collections[0].colorbar   # 获取 colorbar 对象
cbar.ax.tick_params(labelsize=18)        # 设置色条刻度字体大小

# 设置色条刻度的小数格式（保留两位小数）
cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

# 保存为 PDF 文件
plt.savefig("heatmap.pdf", format="pdf", bbox_inches="tight")
plt.show() 

# FT-model

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 设置 Seaborn 样式和 matplotlib 参数
sns.set_style("whitegrid")
plt.rcParams.update({
    "axes.edgecolor": "black",     
    "axes.linewidth": 1.0,         
    "axes.facecolor": "white",     
    "grid.color": "gray",          
    "grid.alpha": 0.2,             
    "grid.linestyle": "-",
    "xtick.color": "black",
    "ytick.color": "black",
    "text.color": "black",
    "font.size": 12,
    "figure.facecolor": "white",
})
sns.set_context("notebook", font_scale=1.2)

# 从 Excel 中读取数据
df = pd.read_excel('./vis/performance/shot.xlsx')  # 请确保文件路径正确
labels = df['model'].tolist()
values = df['Cosine'].tolist()

# 按 Cosine 值升序排序（值最小的在内层，最大的在外层）
sorted_data = sorted(zip(labels, values), key=lambda x: x[1])
sorted_labels, sorted_values = zip(*sorted_data)
max_value = max(sorted_values)
N = len(sorted_labels)

# 使用 GnBu 渐变色调色板生成离散颜色列表
colors = sns.color_palette("GnBu", n_colors=N)

# 极坐标图参数
arc_max_deg = 300                        # 最大弧宽（单位：度），预留60°缺口
arc_max_rad = np.deg2rad(arc_max_deg)      # 转换为弧度
theta_start_deg = 90                     # 固定起始角度（所有环均从此角度开始）
theta_start_rad = np.deg2rad(theta_start_deg)

# 每个环的径向参数
r0 = 0.5             # 最内层环的起始半径
ring_thickness = 0.3 # 每个环的厚度
gap = 0.05           # 环之间的间隙

# 标签字体大小（可调）
label_fontsize = 14

# 创建极坐标图
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=True)

# 绘制每个数据点作为嵌套极坐标柱状图
for i, (label, value) in enumerate(zip(sorted_labels, sorted_values)):
    # 根据数值计算当前环的弧宽（比例关系）
    width_i = (value / max_value) * arc_max_rad  
    # 用于绘制柱状图的中心角（这里用于 bar 的位置）
    theta_i = theta_start_rad + width_i / 2  
    
    # 计算当前环的起始径向位置
    bottom_i = r0 + i * (ring_thickness + gap)
    
    # 绘制极坐标柱状图（环）
    ax.bar(theta_i, ring_thickness, width=width_i, bottom=bottom_i,
           color=colors[i], edgecolor='white', alpha=0.8)
    
    # 固定文本的径向位置
    text_r = bottom_i + ring_thickness / 2
    
    # 在环的起点放置 model 标签
    text_theta_start = theta_start_rad  # 起点位置
    ax.text(text_theta_start, text_r, f"{label}", ha='left', va='center', 
            fontsize=label_fontsize, color='black')
    
    # 在环的终点放置 Cosine 值标签
    text_theta_end = theta_start_rad + width_i  # 终点位置
    ax.text(text_theta_end, text_r, f"{value}", ha='left', va='center', 
            fontsize=label_fontsize, color='black')

# 调整图表显示设置
ax.set_ylim(0, r0 + N * (ring_thickness + gap) + 0.5)
ax.set_xticks([])  # 不显示角度刻度
ax.set_yticks([])  # 不显示径向刻度
ax.spines['polar'].set_visible(False)  # 隐藏极坐标轴线
ax.grid(False)  # 隐藏极坐标网格线

plt.tight_layout()
plt.savefig('nested_polar_bar_chart.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as mticker

# ------------------ 设置风格 ------------------
sns.set_style("whitegrid")
plt.rcParams.update({
    "axes.edgecolor": "black",         
    "axes.linewidth": 1.0,             
    "axes.facecolor": "white",         
    "grid.color": "gray",              
    "grid.alpha": 0.2,                 
    "grid.linestyle": "-",
    "xtick.color": "black",
    "ytick.color": "black",
    "text.color": "black",
    "font.size": 12,
    "figure.facecolor": "white",
})
sns.set_context("notebook", font_scale=1.2)

# ------------------ 使用 GnBu 渐变色调色板 ------------------
cmap = sns.color_palette("GnBu", as_cmap=True)

# ------------------ 读取 Excel 文件 ------------------
df = pd.read_excel("./vis/performance/structure.xlsx", index_col=0)
df = df.astype(float)

# ------------------ 分组列名 ------------------
group1 = ["Metal Source", "Organic Linkers Source", "Modulator Source", "Solvent Source",
          "Quantity of Metal", "Quantity of Organic Linkers", "Quantity of Modulator", "Quantity of Solvent"]
group2 = ["pH", "Synthesis Temperature", "Synthesis Time", "Equipment"]
group3 = ["Crystal Morphology", "Yield"]

# ------------------ 合并数据 ------------------
# 直接合并所有列，不添加空列
all_columns = group1 + group2 + group3
selected_df = df[all_columns]

# ------------------ 设置热图大小 ------------------
cell_size = 2
fig_width = len(all_columns) * cell_size
fig_height = len(df.index) * cell_size

# ------------------ 创建图表 ------------------
plt.figure(figsize=(fig_width, fig_height))

# 绘制热图
heatmap = sns.heatmap(selected_df,
                      annot=True,
                      fmt=".2f",
                      cmap=cmap,
                      square=True,
                      linewidths=0.5,
                      linecolor='black',
                      cbar_kws={'label': 'Value'},
                      annot_kws={"fontsize": 16, "ha": "center", "va": "center"})

# 设置标签
plt.xticks(np.arange(len(all_columns)) + 0.5, all_columns, rotation=45, ha='right', fontsize=18)
plt.yticks(np.arange(len(df.index)) + 0.5, df.index, fontsize=18)

# 添加粗白线分隔不同组
plt.axvline(x=len(group1), color='white', linewidth=3)
plt.axvline(x=len(group1) + len(group2), color='white', linewidth=3)

# 添加组标题
group_fontsize = 20
plt.text(len(group1)/2, -0.05, "Chemicals", ha='center', fontsize=group_fontsize)
plt.text(len(group1) + len(group2)/2, -0.05, "Conditions", ha='center', fontsize=group_fontsize)
plt.text(len(group1) + len(group2) + len(group3)/2, -0.05, "Crystallizations", ha='center', fontsize=group_fontsize)

# 调整colorbar
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)
cbar.set_label("Value", fontsize=18)
cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

plt.tight_layout()
plt.savefig("heatmapclass.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Co-reference resolution

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker

# ------------------ 设置图表尺寸和间距参数 ------------------ #
FIGURE_WIDTH = 15          # 图表宽度（英寸）
FIGURE_HEIGHT_PER_PUB = 1  # 每个出版商子图的高度（英寸）
SUBPLOT_WSPACE = 0.2       # 子图之间的水平间距
SUBPLOT_HSPACE = 0.1       # 子图之间的垂直间距
LEFT_MARGIN = 0.15         # 左边距
RIGHT_MARGIN = 0.75        # 右边距（为图例留出空间）
TOP_MARGIN = 0.95          # 上边距
BOTTOM_MARGIN = 0.1        # 下边距

# ------------------ 设置风格 ------------------ #
sns.set_style("whitegrid")
plt.rcParams.update({
    "axes.edgecolor": "black",     
    "axes.linewidth": 1.0,         
    "axes.facecolor": "white",     
    "grid.color": "gray",          
    "grid.alpha": 0.2,             
    "grid.linestyle": "-",
    "xtick.color": "black",
    "ytick.color": "black",
    "text.color": "black",
    "font.size": 12,
    "figure.facecolor": "white",
})
sns.set_context("notebook", font_scale=1.2)

# ------------------ 定义颜色方案 ------------------ #
total_color = '#084384'  # Total线的颜色（用于 t 部分外环）
# 出版商线条颜色列表，按排序后依次分配
pub_colors = ['#1373b2', '#42a6cb', '#77cac5', '#b2e1b9', '#d6efd0']
alpha_value = 0.8        # 不透明度
# 定义外环中非 t 部分颜色，使用 pub_colors 列表中最浅的颜色
overall_non_t_color = pub_colors[-1]

# ------------------ 读取数据 ------------------ #
input_file = "./vis/performance/CR.xlsx"
df = pd.read_excel(input_file, index_col=0)
df['pub'] = df['pub'].fillna("None")

# ------------------ 分组统计 ------------------ #
# 针对 pub 分组，统计该组的总数，以及该组中 Label 为 "t" 的个数
grouped = df.groupby('pub').agg(
    total=('pub', 'size'),
    t_count=('Label', lambda x: (x == 't').sum())
).reset_index()
grouped['non_t'] = grouped['total'] - grouped['t_count']
grouped['t_percent'] = grouped['t_count'] / grouped['total'] * 100

# 排除 Total 行（假设 Total 行中 pub 的值为 "Total"）
grouped_no_total = grouped[grouped['pub'] != 'Total']

# 计算总体指标（所有出版商的总数和 t 数量）
overall_total = grouped_no_total['total'].sum()
overall_t = grouped_no_total['t_count'].sum()
overall_non_t = overall_total - overall_t
overall_t_pct = overall_t / overall_total * 100

# 获取需要排序的值和标签
labels = grouped_no_total['pub'].tolist()
values = grouped_no_total['total'].tolist()

# 按总数升序排序（值最小的在内层，最大的在外层）
sorted_data = sorted(zip(labels, values, range(len(labels))), key=lambda x: x[1])
sorted_labels, sorted_values, original_indices = zip(*sorted_data)
sorted_grouped = grouped_no_total.iloc[list(original_indices)].reset_index(drop=True)

N = len(sorted_labels)
max_value = max(sorted_values)

# 使用 GnBu 渐变色调色板生成离散颜色列表 - 按排序后的顺序分配颜色
base_colors = sns.color_palette("GnBu", n_colors=N)
t_colors = sns.color_palette("GnBu_r", n_colors=N)

# ------------------ 极坐标图参数 ------------------ #
arc_max_deg = 300                        # 最大弧宽（单位：度），预留60°缺口
arc_max_rad = np.deg2rad(arc_max_deg)     # 转换为弧度
theta_start_deg = 90                     # 固定起始角度（所有环均从此角度开始）
theta_start_rad = np.deg2rad(theta_start_deg)

# 每个环的径向参数
r0 = 1.0             # 最内层环的起始半径
ring_thickness = 1   # 每个出版商环的厚度
gap = 0.15           # 环之间的间隙

label_fontsize = 20

# ------------------ 创建极坐标图 ------------------ #
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, polar=True)

# 绘制每个出版商的环（从内到外）
for i, row in sorted_grouped.iterrows():
    pub_name = row['pub']
    total = row['total']
    t_count = row['t_count']
    t_pct = row['t_percent']
    
    # 当前环的弧宽与该出版商总数成正比
    width_i = (total / max_value) * arc_max_rad
    theta_i = theta_start_rad + width_i / 2  # bar 中心角
    bottom_i = r0 + i * (ring_thickness + gap)
    
    # 绘制 non-t 部分
    ax.bar(theta_i, ring_thickness, width=width_i, bottom=bottom_i,
           color=base_colors[i], edgecolor='white', alpha=alpha_value, linewidth=1)
    
    # 绘制 t 部分（堆积在上半部）
    if t_count > 0:
        t_width = width_i * (t_count / total)
        t_theta = theta_start_rad + t_width / 2
        ax.bar(t_theta, ring_thickness/2, width=t_width, bottom=bottom_i + ring_thickness/2,
               color=t_colors[i], edgecolor='white', alpha=alpha_value, linewidth=1)
    
    # 在环的起始处添加出版商名称
    text_r = bottom_i + ring_thickness / 2
    ax.text(theta_start_rad, text_r, f"{pub_name}", ha='left', va='center',
            fontsize=label_fontsize, color='black')
    # 在环的终点处添加总数和 t 百分比
    ax.text(theta_start_rad + width_i, text_r, f"{total} ({t_pct:.1f}%)", 
            ha='left', va='center_baseline', fontsize=label_fontsize, color='black')

# 绘制外部大环（总体统计）
outer_bottom = r0 + N * (ring_thickness + gap)
outer_thickness = ring_thickness  # 外环厚度可与内部相同
# 外环总宽度设为完整的360°（2π）
outer_total_width = 2 * np.pi
# 根据总体数据计算非-t和t部分的弧宽
outer_non_t_width = outer_total_width * (overall_non_t / overall_total)
outer_t_width = outer_total_width * (overall_t / overall_total)
theta_non_t_center = theta_start_rad + outer_non_t_width/2
theta_t_center = theta_start_rad + outer_non_t_width + outer_t_width/2

ax.bar(theta_non_t_center, outer_thickness, width=outer_non_t_width, bottom=outer_bottom,
       color=overall_non_t_color, edgecolor='white', alpha=alpha_value, linewidth=1)
ax.bar(theta_t_center, outer_thickness, width=outer_t_width, bottom=outer_bottom,
       color=total_color, edgecolor='white', alpha=alpha_value, linewidth=1)

# 添加总体比例文本
outer_text_r = outer_bottom + outer_thickness/2
ax.text(theta_start_rad, outer_text_r, f"Overall\n{overall_total} ({overall_t_pct:.1f}%)", 
        ha='left', va='center', fontsize=label_fontsize, color='black')

# 调整图表显示设置
ax.set_ylim(0, r0 + (N+1) * (ring_thickness + gap) + 0.5)
ax.set_xticks([])  # 隐藏角度刻度
ax.set_yticks([])  # 隐藏径向刻度
ax.spines['polar'].set_visible(False)  # 隐藏极坐标轴线
ax.grid(False)  # 隐藏极坐标网格线

fig.tight_layout()
plt.savefig('cr.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Paraparagraph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as mticker

# ------------------ 设置图表尺寸和间距参数 ------------------ #
FIGURE_WIDTH = 15  # 图表宽度，英寸
FIGURE_HEIGHT_PER_PUB = 1  # 每个出版商子图的高度，英寸
SUBPLOT_WSPACE = 0.2  # 子图之间的水平间距
SUBPLOT_HSPACE = 0.1  # 子图之间的垂直间距
LEFT_MARGIN = 0.15  # 左边距
RIGHT_MARGIN = 0.75  # 右边距，进一步调小为图例留出更多空间
TOP_MARGIN = 0.95  # 上边距
BOTTOM_MARGIN = 0.1  # 下边距

# ------------------ 设置风格 ------------------ #
sns.set_style("whitegrid")
plt.rcParams.update({
    "axes.edgecolor": "black",  # 黑色边框
    "axes.linewidth": 1.0,      # 边框宽度
    "axes.facecolor": "white",
    "grid.color": "gray",
    "grid.alpha": 0.2,
    "grid.linestyle": "-",
    "xtick.color": "black",
    "ytick.color": "black",
    "text.color": "black",
    "font.size": 12,
    "figure.facecolor": "white",
})
sns.set_context("notebook", font_scale=1.2)

# ------------------ 定义颜色方案 ------------------ #
total_color = '#084384'  # Total线的颜色
pub_colors = ['#1373b2', '#42a6cb', '#77cac5', '#b2e1b9', '#d6efd0']  # 出版商线条的颜色列表
alpha_value = 1  # 设置80%不透明度（20%透明度）

# ------------------ 读取数据 ------------------ #
df = pd.read_excel("./vis/performance/parag.xlsx")  # 包含列: pub, 25-shot, 50-shot, 75-shot, 100-shot

# 找到并提取 "total" 行数据
total_data = df[df['pub'] == 'Total'].iloc[0]  # 假设只有一行 Total

# 提取不包括 Total 行的其他出版商数据
publishers_df = df[df['pub'] != 'Total']

# 将数据转换为长格式
df_long = pd.melt(publishers_df, id_vars=['pub'], var_name='shot', value_name='value')

# 提取 shot 数字，用于排序 & 绘图
df_long['shot_num'] = df_long['shot'].str.extract('(\d+)').astype(int)
df_long = df_long.sort_values('shot_num')

# 获取 pub 的唯一值（出版商名称）
unique_pub = df_long['pub'].unique()

# 计算每个出版商的平均值，用于确定颜色顺序
pub_averages = {}
for pub in unique_pub:
    pub_data = df_long[df_long['pub'] == pub]
    pub_averages[pub] = pub_data['value'].mean()

# 根据平均值从高到低排序出版商
sorted_pubs = sorted(pub_averages.items(), key=lambda x: x[1], reverse=True)
sorted_pub_names = [item[0] for item in sorted_pubs]

# 为每个出版商分配颜色 - 值最高的使用第一个颜色，依此类推
pub_color_map = {}
for i, pub in enumerate(sorted_pub_names):
    color_idx = min(i, len(pub_colors) - 1)  # 防止索引超出颜色列表范围
    pub_color_map[pub] = pub_colors[color_idx]

# 创建 x 轴标签和刻度
shot_columns = ['25-shot', '50-shot', '75-shot', '100-shot']
shot_nums = [int(col.split('-')[0]) for col in shot_columns]

# ------------------ 创建子图（小 multiples） ------------------ #
# 根据出版商数量和每个出版商的高度计算总高度
total_height = FIGURE_HEIGHT_PER_PUB * len(unique_pub)
fig, axes = plt.subplots(nrows=len(unique_pub), ncols=1, sharex=True, 
                        figsize=(FIGURE_WIDTH, total_height))

# 设置子图之间的间距
plt.subplots_adjust(wspace=SUBPLOT_WSPACE, hspace=SUBPLOT_HSPACE, 
                    left=LEFT_MARGIN, right=RIGHT_MARGIN, 
                    top=TOP_MARGIN, bottom=BOTTOM_MARGIN)

# 创建一个字典存储每个出版商和Total的图例句柄和标签
legend_handles = []
legend_labels = []

# 遍历每个出版商，分别绘制到对应子图
for i, pub_name in enumerate(unique_pub):
    ax = axes[i] if len(unique_pub) > 1 else axes  # 当只有一个出版商时，axes 不是列表
    
    # 筛选当前出版商的数据
    subset = df_long[df_long['pub'] == pub_name]
    
    # 使用为该出版商分配的颜色（基于其平均值排名）
    pub_color = pub_color_map[pub_name]
    
    # 绘制当前出版商的线条，使用分配的颜色和指定的透明度
    line1, = ax.plot(subset['shot_num'], subset['value'], marker='o', markersize=6, 
                    color=pub_color, alpha=alpha_value, label=pub_name)
    
    # 在同一子图中绘制 Total 数据作为参考线，使用指定的蓝色
    line2, = ax.plot(shot_nums, [total_data[col] for col in shot_columns], 
                    marker='s', markersize=5, color=total_color, alpha=alpha_value, 
                    linestyle='--', label='Total')
    
    # 仅在第一个子图时收集Total的图例信息
    if i == 0:
        legend_handles.append(line2)
        legend_labels.append('Total')
    
    # 收集每个出版商的图例信息
    legend_handles.append(line1)
    legend_labels.append(pub_name)
    
    # 设置 Y 轴刻度格式为保留两位小数
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    
    # 只在中间位置的子图添加Y轴标签，并旋转90度
    if i == len(unique_pub) // 2:  # 中间位置的子图
        ax.set_ylabel("Cosine similarity", rotation=90, labelpad=15, va='center', fontsize=18)
    else:
        ax.set_ylabel("")  # 其他子图不显示Y轴标签
    
    # 只在最后一个子图显示 X 轴刻度和标签
    if i < len(unique_pub) - 1:
        ax.set_xticklabels([])  # 隐藏 X 轴刻度标签
    else:
        ax.set_xlabel("Shot number")
        ax.set_xticks(shot_nums)
        ax.set_xticklabels(shot_columns)
    
    # 确保显示四边黑框
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1.0)
    
    # 移除单个子图的图例
    if ax.get_legend():
        ax.get_legend().remove()

# 在图的右侧添加所有图例，使用bbox_to_anchor将图例放在轴外
fig.legend(legend_handles, legend_labels, loc='center left', 
           bbox_to_anchor=(1, 0.5), frameon=False)

fig.tight_layout()
plt.savefig('parag.pdf', dpi=300, bbox_inches='tight')
plt.show()